In [ ]:
from pathlib import Path

import pandas as pd

In [ ]:
seq_status = pd.read_csv("/Users/andrewprokhorenkov/CTDS/projects/midrc/data/sequestration/master_sequestration_locations_54536_2022-10-24.tsv", sep="\t")
open_cases = seq_status[seq_status["dataset"] == "Open"]["case_ids"]

dicom_studies = pd.read_csv("/Users/andrewprokhorenkov/CTDS/projects/midrc/data/dicom/studies.txt", header=None)

In [ ]:
manifests = list(Path(".").glob("*.tsv"))

In [ ]:
manifest_dfs = {f.stem: pd.read_csv(f, sep="\t") for f in manifests}

In [ ]:
manifest_dfs.keys()

In [ ]:
for k, v in manifest_dfs.items():
    print(k)
    df = v
    if k in ["midrc-ricord-2021-09-02", "midrc-ricord-2021-09-22"]:
        manifest = df
    else:
        manifest = df.loc[(df["case_id"].isin(open_cases))]
    # manifest = df

    print(manifest.shape)

    dicom_missing = manifest[~manifest["study_id"].isin(dicom_studies[0])]
    print(dicom_missing.shape)

In [ ]:
manifest_df = pd.concat(manifest_dfs.values(), ignore_index=True)

In [ ]:
manifest = manifest_df.loc[(manifest_df["case_id"].isin(open_cases)) & \
    (~manifest_df["study_id"].isin(dicom_studies[0]))]

In [ ]:
manifest

In [ ]:
manifest.to_csv("manifest.tsv", sep='\t', index=False)

In [ ]:
manifest = pd.read_csv("./manifest.tsv", sep="\t")
manifest["storage_urls"].str.replace("s3://storage.ir.rsna.ai/", "").to_csv("dicom_rsna.lst", index=False, header=None)